$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

$\textbf{Preliminares}$ 

Las librerías que se usan en este capítulo son las siguientes: 

In [ ]:
!rm -rf Uniandes_Framework
!git clone https://github.com/Phenomenology-group-uniandes/Uniandes_Framework.git

import os, sys
sys.path.append(f'{os.getcwd()}/Uniandes_Framework')
personal_folder = os.getcwd()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler #Permite balancear datasets
import xgboost #Gradient Boosting - Modelo de Machine Learning
from sklearn.model_selection import train_test_split #Permite separar los datos en los conjuntos de datos para entrenar y testear el modelo de ML
from sklearn.model_selection import GridSearchCV #Permite optimizar la busqueda de parametros de un modelo de ML
from sklearn.metrics import accuracy_score #Permite calcular la precisión de los modelos de ML

import ml_tools #Herramientas de Machine Learning 

En este capítulo se utilizan la lista signals por lo que es necesario volverla a definir para no tener inconvenientes:

In [ ]:
#Definamos una lista con las señales y un directorio para guardar las secciones eficaces:
signal = "z"
bkgs = ["w_jets", "ww", "wz", "zz", "ttbar", "stop"]

$\textbf{Leer los datos de los archivos .csv}$

In [ ]:
Datasets_correlation = {} #Incluye todas las variables cinemáticas del muon 1 y del muon 2
Datasets_Z = {}

Datasets_correlation[signal] = pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/Data_correlation/{signal}.csv')
del Datasets_correlation[signal][Datasets_correlation[signal].columns[0]]
Datasets_Z[signal] = pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/Data_Z/{signal}.csv')
del Datasets_Z[signal][Datasets_Z[signal].columns[0]]

for bkg in bkgs:
    Datasets_correlation[bkg] = pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/Data_correlation/{bkg}.csv')
    del Datasets_correlation[bkg][Datasets_correlation[bkg].columns[0]]
    Datasets_Z[bkg] = pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/Data_Z/{bkg}.csv')
    del Datasets_Z[bkg][Datasets_Z[bkg].columns[0]]

cut_flows = pd.DataFrame.to_dict(pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/cut_flows.csv',index_col = 0))

Lo primero que debemos hacer es contatenar las columnas de todos los datasets para la señal y para el background. En general si tenemos dos DataFrames:

In [ ]:
Datasets = {}

In [ ]:
Datasets_Z['z']

In [ ]:
Datasets_correlation['z']

En este caso, esas son todas las variables cinemáticas asociadas a la señal Z. Si queremos concatenarlas en solo un DataFrame, para eso hay que usar pd.concat y su parametro axis igualarlo a 1 (cuando vale 0 se concatenan filas).

In [ ]:
Datasets['signal'] = pd.concat([Datasets_Z['z'], Datasets_correlation['z']], axis = 1)

In [ ]:
Datasets['signal']

Ahora hagamos lo mismo para todo el background, concatenemos las columnas y luego mezclemoslo todo en un dataset:

In [ ]:
Datasets['BKG'] = pd.DataFrame()
for bkg in bkgs:
    bkg_con_todas_las_columnas = pd.concat([Datasets_Z[bkg], Datasets_correlation[bkg]], axis = 1) #Se concatenan las columnas
    Datasets['BKG'] = pd.concat([Datasets['BKG'], bkg_con_todas_las_columnas], axis = 0) #Se concatenan todos los bkg en uno de muchas filas    
    
Datasets['BKG'].index = [i for i in range(len(Datasets['BKG']['pT_{Z}(GeV)']))] #Si no se hace esto habrán index repetidos

In [ ]:
Datasets['BKG']

Así, Datasets es un directorio que tiene todas las variables cinematicas de la señal y del background:

In [ ]:
Datasets.keys()

$\textbf{Preparar todos los datos en un solo Dataset}$

Para esto debemos añadir una columna de ceros y unos donde el "1" representa que el evento es señal y el "0" representa que el evento es background en Datasets['signal'] y Datasets['BKG']. Luego de eso debemos concatenarlos ambos en un nuevo Dataset.

In [ ]:
labels_signal = {'Label': [1 for evento in Datasets['signal']['pT_{Z}(GeV)']]}
Datasets['signal'] = pd.concat([Datasets['signal'], pd.DataFrame.from_dict(labels_signal)], axis = 1)

In [ ]:
Datasets['signal']

In [ ]:
labels_bkg = {'Label': [0 for evento in Datasets['BKG']['pT_{Z}(GeV)']]}
Datasets['BKG'] = pd.concat([Datasets['BKG'], pd.DataFrame.from_dict(labels_bkg)], axis = 1)

In [ ]:
Datasets['BKG']

Ahora volvamos esos dos datasets uno gigante.

In [ ]:
Data = pd.concat([Datasets['signal'], Datasets['BKG']], axis = 0)
Data.index = [i for i in range(len(Data['pT_{Z}(GeV)']))] #Si no se hace esto habrán index repetidos

Así,

In [ ]:
Data

Sin embargo, acá estan ordenados, debemos mezclarlos:

In [ ]:
Data = Data.sample(frac=1).reset_index(drop=True)

In [ ]:
Data

$\textbf{Balancear Dataset (CREO QUE ESTE PASO NO ES NECESARIO)}$

En este punto es claro que hay más filas de BKG que de signal:

In [ ]:
# Data.value_counts('Label')

Para balancear el dataset es necesario escoger 183580 eventos del BKG de manera aleatoria, esto se puede hacer con RandomUnderSampler:

In [ ]:
# rus = RandomUnderSampler()
# Data, Data['Label'] = rus.fit_resample(Data, Data['Label'])

Así,

In [ ]:
# Data.value_counts('Label')

$\textbf{Separar en X y Y}$

Debemos separar el Dataset en X y Y. Es decir, en input y output para que más adelante sea posible entrenar el modelo de Machine Learning. En este caso es claro que el output es la columna 'Label' y el input son todas las demás columnas:

In [ ]:
X = Data.loc[:, Data.columns!= 'Label'] #Todas las filas y sus columnas que sean distintas a 'Label'
Y = Data.loc[:, 'Label']  #Todas las filas de la columna 'target'

Así,

In [ ]:
X

In [ ]:
pd.DataFrame(Y)

$\textbf{Separar conjunto de datos para "train" y para "test"}$

Separemos un 50% del Dataset para entrenar el modelo, un 25% para testearlo y un 25% para validarlo. Para esto podemos hacer uso de la herramienta train_test_split que se importó al inicio de este capítulo.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 1) #El random_state es para que siempre que se corra este notebook se haga igual

In [ ]:
X_train.shape, X_test.shape

Aquí separamos primero en 50-50 con la idea de ahora separar X_train y Y_train otra vez 50-50, es decir un 25-25 del total de datos, esto con el objetivo de poder entrenar y validar el modelo.

In [ ]:
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size = 0.5, random_state = 1)

Así, nos queda:

In [ ]:
X_train.shape,X_test.shape, X_valid.shape # 50 - Entrenar, 25 - Testear y 25 - Validar

In [ ]:
Y_train.shape,Y_test.shape, Y_valid.shape # 50 - Entrenar, 25 - Testear y 25 - Validar

$\textbf{Crear modelo de Machine Learning}$

Para crear un modelo de Gradient Bosting se puede usar xgboost

In [ ]:
xgb = xgboost.XGBClassifier() #Modelo clasificador de gradient boosting 

In [ ]:
xgb.fit(X_train, Y_train)

Aquí ya podemos usarlo para predecir datos, simplemente hay que usar xgb.predict() donde el parametro sera el input que deseemos evaluar. Por ejemplo, usemos X_valid como input

In [ ]:
Y_predict = xgb.predict(X_valid)

In [ ]:
Y_predict

Efectivamente el output es una fila de ceros y unos, comparemos esto con Y_valid que contiene los valores reales de los labels para ver que tan bueno es el modelo:

In [ ]:
pd.DataFrame({'Real': Y_valid, 'Predicción' : Y_predict})

Aquí se puede ver que funciona bien en algunos casos y en otros no tanto. Analicemos esto de una forma más rigurosa.

$\textbf{Evaluemos su precisión}$

Para esto usemos X_Valid y Y_valid:

In [ ]:
accuracy_score(Y_valid, Y_predict)

Si quisieramos un accuracy más grande es necesario analizar los parametros que contiene el modelo de gradient boosting. Ese modelo tiene un conjunto de parámetros, la idea para mejorar la precisión es buscar los que mejor predigan los datos, para esto lo que se hace es darle opciones al modelo y que el analice todas las posibles combinaciones y así extraíga la mejor:

In [ ]:
parameters = {'nthreads': [1], 
             'objective': ['binary:logistic'],
             'learning_rate': [0.05,0.1],
             'n_estimators': [100,200]}

Ahora, usando GridSearcCV se puede entrenar el modelo usando todas las combinaciones posibles de parametros dentro de parameters, esta herramienta buscará la que mejor se ajusta. No obstante, esto se podría demorar bastante, para mejorar su tiempo de ejecución anteriormente dejamos un 25% de los datos para testear, la idea es con ese de 25% definir una función de costo que permita testear cada posible combinación de los parameters, así si en cierta cantidad de rondas la función de costo no mejora entonces se analiza con la siguiente combinación posible de parametros.

In [ ]:
fit_params = {'early_stopping_rounds': 10,
             'eval_metric': 'logloss',
             'eval_set': [(X_test, Y_test)] }

Ese diccionario significa que va a analizar hasta la ronda 10 los entrenamientos usando X_test y Y_test evaluados en la función de costo, si en 10 rondas la función de costo no mejora entonces el entrenamiento para, así podemos mejorar el tiempo de ejecución que use GridSearchCV para encontrar los mejores parametros dentro de parameters.

In [ ]:
clf = GridSearchCV(xgb, parameters, cv=3, scoring='accuracy') #cv=3 es hacer cross validation 3 veces
clf.fit(X_train, Y_train, **fit_params)

In [ ]:
Best_xgb = clf.best_estimator_ #Ese es el modelo con la mejor combinación de parametros

In [ ]:
Best_xgb

Evaluemos su precisión usando X_Valid y Y_valid:

In [ ]:
Y_predict = Best_xgb.predict(X_valid)

In [ ]:
accuracy_score(Y_valid, Y_predict)

Mejoró un poco, así que como se quería es un mejor modelo. Esto se podría mejorar aún más si se pone un mayor número de arboles (n_estimators). La idea es jugar con todos los parametros.

$\textbf{Grafiquemos su distribución de probabilidad (Score)}$

Cada modelo de ML tiene una grafica de Score asociada:

In [ ]:
Dist = Best_xgb.predict_proba(X_valid)

In [ ]:
Dist #Contiene para cada valor de X_valid la probabilidad de que su output sea 0 y 1 respectivamente.

In [ ]:
plt.hist(Dist[:,0], label = 'P(BKG)', bins = 50, alpha = 0.5)
plt.hist(Dist[:,1], label = 'P(Signal)', bins = 50, alpha = 0.5)

plt.xlim(0,1)
plt.xlabel('ML-Output')
plt.ylabel('N events')
plt.title('Score Distribution')
plt.legend()
plt.show()

Para cada valor de X (ML_Output) se puede calcular una significancia.

$$ \textbf{Significancia} = \frac{\textrm{Número de Datos de señal}}{ \sqrt{\textrm{Número de Datos de señal + Número de Datos de Background}}} $$

Dado que tenemos la probabilidad simplemente habría que multiplicarla por el número total de eventos y así tendríamos el número de eventos de señal y de background respectivamente.

In [ ]:
N_events = len(Dist) #Numero total de eventos

Significances = np.array([])

for i in range(N_events):
    N_Signal = N_events*Dist[1]
    N_BKG = N_events*Dist[0]
    
    Significance = N_Signal / np.sqrt(N_Signal + N_BKG)
    
    Significances = np.append(Significances, Significance)

In [ ]:
Significances

Busquemos el máximo para saber cuales son los valores de P(BKG) y P(Signal) que maximizan la significancia.

In [ ]:
Dist[np.argmax(Significances)]

Ese sería el mejor corte para calcular la significancia.

$\textbf{Nota:}$ Todo esto se hizó sin recurrir a Pheno_BSM debido a que el código escrito en la carpeta ml_tools está escrito para una estructura de datos particular que no fue la que se uso en estos tutoriales. Sin embargo, si se entiende esto bien no debería haber problema en entender que hace cada función dentro de ml_tools.